# Outline

## Training and Tuning ML Models
- Hyperparameter Tuning
    - Algorithms for model tunign
    - Sagemaker automated parameter tuning
    - BERT Automated tuning
    
    - warm start hyperparameter turning
        - speedup the tuning process
    - Check point in machine learning
        - managed spot training; to reduce the cost of training
        
    - Distirbuted training challenges
        - Tune distributed training stragegies
            - data panelism
            - model panelism
            
    - Bring your own container

# Advanced Model Training and Tuning

<img src='./pics/courser outline.png'>

- challenges in training and tuning large machine learning models while optimizing cost

- Popular algorithms for hyperparameter tuning (Automatic)
    - 4 algorithms
    - <u>Grid search</u>
        - specify parameters to tune
        - specify the range of the values for each parameter
        - the algorithm will train the model of every combination and find the best parameter

        - Pros
            - Explore all possible combintions
            - works for small number of parameters
        - Cons
            - time consuming for large number of parameters
            - doesn't scale for large number of parameters
        
    - <u>Random search</u>
        - Specify the hyperparmter to tune
        - Define the search space and stop criteria
            - some stop criteria may be,
                - time elapsed
                - maximum number of models to be trained
                
        - Test random combinations within the search space
        - select the best combination
        
        - Pros
            - Faster than the grid search
        - Cons
            - May miss the best performing hyperparameters
        
    - <u>Bayesian optimization</u>
        - Hyperparameter is treated as a regression problem
        - Start with random hyperprameters
        - Narrow down the search space around a better performing hyperprameter
        
        - Pros
            - more efficient in finding the best hyperparameter
            
        - Cons
            - requires sequential execution
            - migh get stuck in local minima
            
            - <img src='./pics/local minima.png'>
        
            - this is a prominent probelm when we are using gradient discent for opimizing the probelm
                
    - <u>Hyperband</u>
        - <b>Bandit-based</b> approach (exploitations, explorations)
        - Start from random hyperparameters
        - Explore set of hyperparameters for few iterations (after exploration abandon half of the least performing sets)
        - choose best and explore longer
        - Repeat until max_iterations reached or one candidate left
        
        - Pros
            - Spends time efficiently
        - Cons
            - might discard good candidates early that converge slowly

# Tune a BERT-based Text Classifer

- BERT model Hyper parameter tuning
    - Run multiple training with diff hyperparameters
    - the tunign ob accepts
        - Turning strategy
            - sagemaker natively supports 2 strategies
                - Random
                - Baysian
                
        - objective metrics
            - an example will be maximize the validation accuracy
            
        <img src='./pics/Turning strategy.png'>
        
    - Use case
        - Tune the BERT Text Classifier
            - Tuning strategy
                - Random
            - Objective Metrics
                - Maximize the validation accuracy
                
            - <b>Hyperparametes</b>
                - learning rate?
                - train_batch_size
                
    - The step involves in the process
    
        - create pytorch estimator
            - define fixed hyperparameters that we donot ant to tune during the process
            - create a dictionary
            - <img src='./pics/hyperparameter dictionary.png'>
            
            - pass the fixed hyperparamerters to the estimator
            
        - create HPT(Hyperparameter Tuning) job
            - define the tunable hyperparameres
            - <img src='./pics/tunable hyperparameter dictionary.png'>
            - specified the name of the hpyerparameters and the range of the values, along with that the hyperparameter tyep is also sepcified. The type may be one of the following,
                - categorical
                - continous
                - integer 
                
            - to test values insted of range (the pic `train_batch_size` parameter, specify it as a categorical parameter
            - integer parameter
                - <img src='./pics/integer parameter.png'>
                - all values between 16 and 1024 has to be explored in the above case
            - if the values to be explored is large use logarithamic scale to optimize the process (look at the pic above)
            
            - continous parameter 
                - learning rate may be a continous parameter
                - <img src='./pics/continous parameter.png'>
                
            - Now create the Hyperparameter tuner object
                - <img src='./pics/Hyperparameter tuner object.png'>
                    - estimator is the estimator we defined for training
                    - hyperparameter rage is the tunable hyperparameter defined
                    - give the objective type
                    
            - `tuner.fit(inputs={...}, ...)`
                
        - analyze the results
            - <img src='./pics/tuner result.png'>
            
       - Warm start
           - start from a perviously trained hyperparameter job
               - to change the hyperparameter range
               - explore new set of hyperparameters
               
               - 2 jobs
                   - IDENTICL_DATA_AND_ALGORITHM
                       - same data and algorithm of the parent tuning job
                       - can update the hyperparameter range and the maximum number of tuning jobs
                   - TRANSFER_LEARNING
                       - updated training data and a diff version of the training algorithm
                       - warm start in code
                       - <img src='./pics/warm start code.png'>
                           - warm start config specifies is it an idendital data and code or a trasfer leaning problem in the warm start config 
                           - pass the warm start config to the Hyperparameter tuner job `warm_start_config` parameter


## Best Practices

- select a small number of hyperparameters
    - computational complexity directly prepotional to the number of hyperparamter to tune
        - sagemaker allow upto 20 hyperparameter to tune at a time
- select a small range of values to tune during the tuning
       
- Enable warm smart
- Enable early stop
    - stop the training when the objective metric is not improving
- select small list of concurrent training jobs
    - turning can be done parallel
    - but the turning is a sequenctial process, one job is better than the other can be determined by completing the previous process
    - so choose a small no of concurrent process
    
- Right size compute resources
    - empherical testing
    - coud watch metric for compute resource utalization
    - Debugger will tell the resouce utalization
    

# <u>ML model training challenges</u>

# Checkpointing

- ML training uses checkpoints to reduce the cost of training
- Checkpointing
    - way to save the current state of model during training
    - the training can be resumed from a known state
        - in case of an unforeseen event (os failure)
        
    - checkpoints are the snapshots of the model
        - model archtecture
        - model weights
        - training configureations
            - no of epochs
            - loss and optimizers
            - other metadata
        - optimizer state
        
- Importtant points while checkpointing
    - fequeceny and number of checkpointing
    - storage vs model infromation trade off
    
- <u><b>Amazon SageMaker Managed Spot</b></u>
    - reduce the training cost
    - unused spot instances at a discounted price
    - use spot instances to 
        - train
        - hyperparameter tuning
        - use checkpoints to resume training
    
- How it works
    - start training in a docker container
        - script: train.py
        - the spot instance will stop with 2 minute warning
        - the spot instance should have the ability to save the checkpoint (/opt/ml/checkpoints)
   - backup the docker to an s3
   - ability to resume from the checkpoints
   - <img src='./pics/checkpoints.png'>
        
    - in case the spot instance is terminated, the sagemaker managed spot polls for the spots and once the spot is find the managed spot resumes the trianing and by trasnfering the checkpoints and the data to the new instnace 

# Distributed Training Strategies

- Training at scale
    - increased the data volume
    - increased model size and complexity
    
- it is difficult to fit a very large model into a single computer memory
- distributed compute is a way to address the scale challenges
- <img src='./pics/distributed training.png'>
    
- the load is distirbuted among
    - single computer multiple gpus/cpus
    - muliptle computer multiple gpus/cpus
    
- There are 2 distributed training strategies
    - data parallelism
        - trainging data is split up among the training node
        - model replicated among the nodes
    - model parallelism
        - the model is split up among the nodes
        - data is replicated among the nodes
    - <img src='./pics/distribute strategies.png'>
- code below- 

In [ ]:
# data parallelism
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point = 'train.py',
    role = sagemaker.get_execution_role(),
    framework_version='1.6.0',
    py_version='py3',
    instance_count=3,
    instance_type='ml.p3.16xlarge',
    distribution={'smdistributed':{'dataprallel':{'enabled':True}}}
)

estimator.fit()


# model parallelism
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point = 'train.py',
    role = sagemaker.get_execution_role(),
    framework_version='1.6.0',
    py_version='py3',
    instance_count=3,
    instance_type='ml.p3.16xlarge',
    distribution={'smdistributed':{'modelparellel':{'enabled':True}}}
)

estimator.fit()

- Distiributeion overhead
    - internode communication
    
- when to use diff strategies
    - model fit on a single node
    - use data prallelism
    - model doesnot fit
    - resize the model
        - hyperparameter tuning
            - number of layers 
            - the optimzer to use
            - these two parameters has a considerable effect on the final model size
        - Reduce the batch size
            - incrementally reduce the batch size
        - Reduce the model input size
            - consider the model is taking a text input consider embedding the text with a low dimensional embedding vector
            - in case of a image input, try to reduce the image resolution
    -even after the experimentation md
-
-

# Custom Algorithms with Amazon SageMaker